In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [4]:
from collections import Counter, OrderedDict
import collections

In [5]:
import joblib


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [7]:
import re

In [8]:
from textblob import TextBlob

Based on https://www.kaggle.com/venomsnake/analyzing-sarcasm-comprehensive-eda-detection and https://github.com/surajr/SarcasmDetection

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df_isarcasm=pd.read_csv("/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv")

In [11]:
df_isarcasm_new=df_isarcasm.loc[df_isarcasm['sarcastic']==1]

In [ ]:
df_isarcasm_new

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
862,862,yo @claires do yall do hysterectomies?,1,"Claires, you should not do full hysterectomies.",1.0,0.0,0.0,0.0,0.0,1.0
863,863,@JacobWohlReport Do I need to aquire a wife be...,1,A lot of people don't have a wife.,1.0,0.0,0.0,0.0,0.0,1.0
864,864,I get a lot of boy who cried wolf vibes from t...,1,The red cross is always needy.,0.0,1.0,0.0,0.0,0.0,0.0
865,865,Update: holding hands with your mom and walkin...,1,Holding hands with your parent while walking a...,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_isarcasm_new=df_isarcasm_new[['tweet','sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']]

In [13]:
train, validate, test = \
              np.split(df_isarcasm_new.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm_new)), int(.8*len(df_isarcasm_new))])

In [14]:
train_new=pd.concat([train, validate], ignore_index=True)

In [15]:
test=pd.read_csv('/content/drive/MyDrive/ISarcasm/Test_dataset/taskB.En.input.csv')

In [16]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [17]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [18]:
import spacy

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [21]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [22]:
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet


In [23]:
import sys
from collections import defaultdict

from pprint import pprint

_do_print_debug_info = False

def _print_table(rows):
    col_widths = [max(len(s) for s in col) for col in zip(*rows)]
    fmt = ' '.join('%%-%ds' % width for width in col_widths)
    rows.insert(1, ['─' * width for width in col_widths])
    for row in rows:
        # Uncomment this version to see code points printed out (for debugging).
        # print(list(map(hex, map(ord, list(fmt % tuple(row))))))
        print(fmt % tuple(row))

def _start_end(arrow):
    start, end = arrow['from'].i, arrow['to'].i
    mn = min(start, end)
    mx = max(start, end)
    return start, end, mn, mx

def print_parse_info(nlp, sent):
    """ Print the dependency tree of `sent` (sentence), along with the lemmas
        (de-inflected forms) and parts-of-speech of the words.

        The input `sent` is expected to be a unicode string (of type unicode in
        Python 2; of type str in Python 3). The input `nlp` (for natural
        language parser) is expected to be the return value from a call to
        spacy.load(), in other words, it's the callable instance of a spacy
        language model.
    """

    unicode_type = unicode if sys.version_info[0] < 3 else str
    assert type(sent) is unicode_type

    # Parse our sentence.
    doc = nlp(sent)

    # Build a list of arrow heights (distance from tokens) per token.
    heights = [[] for token in doc]

    # Build the arrows.

    # Set the from and to tokens for each arrow.
    arrows = [{'from': src, 'to': dst, 'underset': set()}
              for src in doc
              for dst in src.children]

    # Set the base height; these may increase to allow room for arrowheads after this.
    arrows_with_deps = defaultdict(set)
    for i, arrow in enumerate(arrows):
        if _do_print_debug_info:
            print('Arrow %d: "%s" -> "%s"' % (i, arrow['from'], arrow['to']))
        num_deps = 0
        start, end, mn, mx = _start_end(arrow)
        for j, other in enumerate(arrows):
            if arrow is other:
                continue
            o_start, o_end, o_mn, o_mx = _start_end(other)
            if ((start == o_start and mn <= o_end <= mx) or
                (start != o_start and mn <= o_start <= mx)):
                num_deps += 1
                if _do_print_debug_info:
                    print('%d is over %d' % (i, j))
                arrow['underset'].add(j)
        arrow['num_deps_left'] = arrow['num_deps'] = num_deps
        arrows_with_deps[num_deps].add(i)

    if _do_print_debug_info:
        print('')
        print('arrows:')
        pprint(arrows)

        print('')
        print('arrows_with_deps:')
        pprint(arrows_with_deps)

    # Render the arrows in characters. Some heights will be raised to make room for arrowheads.

    lines = [[] for token in doc]
    num_arrows_left = len(arrows)
    while num_arrows_left > 0:

        assert len(arrows_with_deps[0])

        arrow_index = arrows_with_deps[0].pop()
        arrow = arrows[arrow_index]
        src, dst, mn, mx = _start_end(arrow)

        # Check the height needed.
        height = 3
        if arrow['underset']:
            height = max(arrows[i]['height'] for i in arrow['underset']) + 1
        height = max(height, 3, len(lines[dst]) + 3)
        arrow['height'] = height

        if _do_print_debug_info:
            print('')
            print('Rendering arrow %d: "%s" -> "%s"' % (arrow_index,
                                                        arrow['from'],
                                                        arrow['to']))
            print('  height = %d' % height)

        goes_up = src > dst

        # Draw the outgoing src line.
        if lines[src] and len(lines[src]) < height:
            lines[src][-1].add('w')
        while len(lines[src]) < height - 1:
            lines[src].append(set(['e', 'w']))
        if len(lines[src]) < height:
            lines[src].append({'e'})
        lines[src][height - 1].add('n' if goes_up else 's')

        # Draw the incoming dst line.
        lines[dst].append(u'►')
        while len(lines[dst]) < height:
            lines[dst].append(set(['e', 'w']))
        lines[dst][-1] = set(['e', 's']) if goes_up else set(['e', 'n'])

        # Draw the adjoining vertical line.
        for i in range(mn + 1, mx):
            while len(lines[i]) < height - 1:
                lines[i].append(' ')
            lines[i].append(set(['n', 's']))

        # Update arrows_with_deps.
        for arr_i, arr in enumerate(arrows):
            if arrow_index in arr['underset']:
                arrows_with_deps[arr['num_deps_left']].remove(arr_i)
                arr['num_deps_left'] -= 1
                arrows_with_deps[arr['num_deps_left']].add(arr_i)

        num_arrows_left -= 1

    arr_chars = {'ew'  : u'─',
                 'ns'  : u'│',
                 'en'  : u'└',
                 'es'  : u'┌',
                 'enw' : u'┴',
                 'ensw': u'┼',
                 'esw' : u'┬'}

    # Convert the character lists into strings.
    max_len = max(len(line) for line in lines)
    for i in range(len(lines)):
        lines[i] = [arr_chars[''.join(sorted(ch))] if type(ch) is set else ch for ch in lines[i]]
        lines[i] = ''.join(reversed(lines[i]))
        lines[i] = ' ' * (max_len - len(lines[i])) + lines[i]

    # Compile full table to print out.
    rows = [['Dep tree', 'Token', 'Dep type', 'Lemma', 'Part of Sp']]
    for i, token in enumerate(doc):
        rows.append([lines[i], token, token.dep_, token.lemma_, token.pos_])
    _print_table(rows)

In [24]:
synonyms = []
antonyms = []
def antysyn(word):
  
    
    from nltk.corpus import wordnet
    for syn in wordnet.synsets(word):
        #print(syn)
        for l in syn.lemmas():
            #print(l)
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    #print("Synonym:",set(synonyms))
    print("Antonym:",set(antonyms))

In [25]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [26]:
#Function for checking negation
def checknegationcontradiction(antonym_tracker,negdoc1,negdoc2):
    temp_var=negdoc1+negdoc2+antonym_tracker
    if(temp_var%2!=0 and temp_var<3):
        return 1
    else:
        return 0

In [27]:
#Finding numerical mismatch
checklist_more=['more than ', 'greater than ', 'above']
checklist_less=['less than ', 'lesser than ', 'below']

In [28]:
def check_words(doc):
    merged_word=""
    #tokens = word_tokenize(doc)
    WordNum  = len(doc)
    print("WordNum is:"+str(WordNum))
    for i in range(WordNum):
        print(i,doc[i],doc[i].ent_type_)
        print(str.isdigit(doc[i].text))
        if(doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM"):
             merged_word = doc[i].text
        if((doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM") and str.isdigit(doc[i].text)):
            if(not(str.isdigit(doc[i-1].text))):
                if(not(str.isdigit(doc[i-2].text))):
                    merged_word= doc[i-2].text+' '+doc[i-1].text+' '+doc[i].text
    return merged_word 

In [29]:
def check_values(t1,t2):
    for phrase in checklist_more:
        if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
            num1 = t1.replace(phrase,'')
            num2 = [int(s) for s in str.split(t2) if s.isdigit()]
            num2=num2[0]
            if int(num1)>num2:
                print("case1")
                return True
            else:
                return False
        else:
            return True
    for phrase in checklist_more:
        if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
            num2 = t2.replace(phrase,'')
            num1 = [int(s) for s in str.split(t2) if s.isdigit()]
            num1=num1[0]
            if int(num2)>num1:
                print("case2")
                return True
            else:
                return False
        else:
            return False
    for phrase in checklist_less:
        if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
            num1 = t1.replace(phrase,'')
            num2 = [int(s) for s in str.split(t2) if s.isdigit()]
            num2=num2[0]
            if int(num1)<num2:
                print("case3")
                return True
            else:
                return False
        else:
                return False
    for phrase in checklist_less:
        if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
            num2 = t2.replace(phrase,'')
            num1 = [int(s) for s in str.split(t2) if s.isdigit()]
            num1=num1[0]
            if int(num1)>num2:
                print("case4")
                return True
            else: 
                return False
        else:
                return False
    else:
        if(t1!=t2):
            return True

In [30]:
def get_Antonymity_negativity(sent):
  global synonyms,antonyms 
  sent_list=split_into_sentences(sent)
  for i in range(0,len(sent_list)):
    for j in range(i+1,len(sent_list)):
      antonyms=[]
      synonyms=[]
      wrdlist=list()
      antony=list()
      contr_tracker=0
      antonym_tracker=0

      negdoc1=0
      negdoc2=0
      verb1=""
      verb2=""
      num_contr_tracker=0
      doc1=nlp(sent_list[i])
      doc2=nlp(sent_list[j])
      #Processing sentence 1
      for token in doc1:
          if(token.dep_=="neg"):
              negdoc1=1
              verb1+="NOT "
          #Storing the antonyms of root words
          if(token.pos_=="VERB" and token.dep_=="ROOT"):
              print(token.text)
              verb1+=token.lemma_
              antysyn(token.lemma_)
              for anton in antonyms:
                  antony.append(anton)
      #Processing Sentence 2
      for token in doc2:
          if(token.dep_=="neg"):
              negdoc2=1
              verb2+="NOT "
          if(token.pos_=="VERB" and token.dep_=="ROOT"):
              verb2+=token.lemma_
              if(token.lemma_ in antony):
                  antonym_tracker=1
      #Checking contradiction due to negation
      contr_tracker=checknegationcontradiction(antonym_tracker,negdoc1,negdoc2)
      x1 = check_words(doc1)
      y1 = check_words(doc2)
      # number_contr_tracker=check_values(x1,y1)
      # if(number_contr_tracker==True):
      #     num_contr_tracker=1
      # else:
      #     num_contr_tracker=0
      if contr_tracker==1:
          return True
      #     print("->Antonymity/Negation contradiction FOUND.")
      # else:
      #     print("\n->Antonymity/Negation contradiction NOT found.")

      if num_contr_tracker==1:
        return True
      #     print("->Numeric Mismatch Contradiction FOUND.")
      # else:
      #     print("->Numeric Mismatch Contradiction NOT Found.")

  return False

In [ ]:
# def get_numeric_mismatch(sent):
#   global synonyms,antonyms 
#   sent_list=split_into_sentences(sent)
#   for i in range(0,len(sent_list)):
#     for j in range(i+1,len(sent_list)):
#       antonyms=[]
#       synonyms=[]
#       wrdlist=list()
#       antony=list()
#       contr_tracker=0
#       antonym_tracker=0

#       negdoc1=0
#       negdoc2=0
#       verb1=""
#       verb2=""
#       num_contr_tracker=0
#       doc1=nlp(sent_list[i])
#       doc2=nlp(sent_list[j])
#       #Processing sentence 1
#       for token in doc1:
#           if(token.dep_=="neg"):
#               negdoc1=1
#               verb1+="NOT "
#           #Storing the antonyms of root words
#           if(token.pos_=="VERB" and token.dep_=="ROOT"):
#               print(token.text)
#               verb1+=token.lemma_
#               antysyn(token.lemma_)
#               for anton in antonyms:
#                   antony.append(anton)
#       #Processing Sentence 2
#       for token in doc2:
#           if(token.dep_=="neg"):
#               negdoc2=1
#               verb2+="NOT "
#           if(token.pos_=="VERB" and token.dep_=="ROOT"):
#               verb2+=token.lemma_
#               if(token.lemma_ in antony):
#                   antonym_tracker=1
#       #Checking contradiction due to negation
#       # contr_tracker=checknegationcontradiction(antonym_tracker,negdoc1,negdoc2)
#       x1 = check_words(doc1)
#       y1 = check_words(doc2)
#       number_contr_tracker=check_values(x1,y1)
#       if(number_contr_tracker==True):
#           num_contr_tracker=1
#       else:
#           num_contr_tracker=0
#       # if contr_tracker==1:
#       #     return True
#       #     print("->Antonymity/Negation contradiction FOUND.")
#       # else:
#       #     print("\n->Antonymity/Negation contradiction NOT found.")

#       if num_contr_tracker==1:
#         return True
#       #     print("->Numeric Mismatch Contradiction FOUND.")
#       # else:
#       #     print("->Numeric Mismatch Contradiction NOT Found.")

#   return False

In [31]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [32]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 1.9 MB/s 


In [33]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [34]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [35]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [36]:
import csv

In [37]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [38]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [39]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [40]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = count
    # print (count)

In [41]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [42]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [43]:
porter = nltk.PorterStemmer()


In [44]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [45]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [46]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train_new['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [47]:
!pip install emoji

     |████████████████████████████████| 174 kB 4.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=5d607ff8470ecd005b6399d9740f92ee9209519c461d248c27142d1b0a3af97e
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [48]:
from emoji import UNICODE_EMOJI



In [49]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [50]:
def num_dashes(headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return  str(headline).count('-') + str(headline).count('–')

In [51]:
def num_possessive( headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return headline.count("'s")

def num_hyphens(headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return  headline.count("-")

def num_commas(headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return headline.count(",")

def num_colons( headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return headline.count(":")

def num_semicolons( headline):
    """
    :param headline: headline to parse
    :return: dict of features for that headline
    """
    headline=str(headline)
    return  headline.count(";")

In [52]:
def ampersands( headline):
      headline=str(headline)
      return headline.count('&')

def percents( headline):
    headline=str(headline)
    return headline.count('%')

def dollars(headline):
    headline=str(headline)
    return  headline.count('$')
def parens( headline):
    headline=str(headline)
    return  headline.count('(') + headline.count(')') + headline.count('[') + headline.count(']')

In [53]:
def quotes( headline):
    headline=str(headline)
    return  headline.count('"') + headline.count("'") + headline.count('`') 

In [54]:
def get_count_interjections(text):
  text=str(text)
  count=0
  doc=nlp(text)
  for word in doc:
    if spacy.explain(word.tag_)=='interjection':
      count=count+1
  return count


In [55]:
def extract_mixed_cpatilization(text):
        """Extracts simple punctuation and capitalisation count-based features from documents
        :param corpus_list: list of documents
        :param doc_name_to_id_dict: dictionary mapping from document name to position in corpus_list
        :returns: dictionary mapping document name to a list of token features
        :rtype: dictionary
        """
        '''
        Go through the documents and extract simple punctuation and lexical 
        features (capitalisation, count of punctuation)
        '''
        text=str(text)
       
        sentences=text.split(".")
        upper_count=0
        lower_count=0
        mixed_count=0
        punctuation_count=0
        for sentence in sentences:
                words=sentence.split(" ")
                for word in words:
                    if word.isupper():
                            if re.search('[a-zA-Z]',word):
                                    upper_count+=1
                    if word.islower():
                            if re.search('[a-zA-Z]',word):
                                    lower_count+=1
                    if not word.islower() and not word.isupper():
                            if re.search('[a-zA-Z]',word):
                                    mixed_count+=1                     
                    if word in string.punctuation:
                            if len(word)>0:
                                    punctuation_count+=1
                            
        
        return mixed_count

In [56]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [57]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [58]:
train_new.dropna(inplace=True)

In [ ]:
train_new

,tweet,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,All the shade i have been hearing about Ben Pl...,1.0,0.0,0.0,0.0,0.0,0.0
1,you would think the odds of me sitting next to...,1.0,0.0,0.0,0.0,0.0,0.0
2,Billy Gunn sorta relevant for the first time i...,1.0,0.0,0.0,1.0,0.0,0.0
3,"I see Dettol, Toilet Duck and Zoflora are all ...",1.0,0.0,0.0,0.0,0.0,1.0
4,@catboychika Aren't you happy I gave you less ...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
688,omg my ships are gonna love what taylor swift ...,1.0,0.0,0.0,0.0,0.0,0.0
689,Maybe if I watched suits in year 12 I’d be stu...,1.0,0.0,0.0,0.0,0.0,0.0
690,"Me: ""Did you you find everything you were look...",1.0,0.0,0.0,0.0,0.0,1.0
691,Broke: doing assignments in order of due date\...,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[0].values

array([1., 0., 0., 0., 0., 0.])

In [59]:
import re
featureset=[]
for i in range(0,len(train_new)):
    # if i==2774or i==2775:
    #   continue
    temp = str(train_new["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values, train_new["tweet"][i]))

In [60]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [61]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0.200000,0.540000,0.204833,0.196862,0.007971,0.000000,0.00000,0.281642,0.169893,0.111748,0.200000,0.540000,9,0.956739
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0.020833,0.156250,0.343094,0.245032,0.098062,0.020833,0.15625,0.274382,0.246909,0.027473,0.000000,0.000000,1,0.947632
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00000,0.229167,0.000000,0.229167,0.250000,0.333333,2,0.875625
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,-0.033333,0.000000,0.046875,0.036458,0.010417,-0.033333,0.00000,0.064560,0.072802,-0.008242,0.000000,0.000000,10,0.917083
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,-0.483333,0.483333,0.000000,0.000000,0.000000,0.000000,0.00000,0.041667,0.316667,-0.275000,-0.483333,0.483333,2,0.889444


In [62]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [63]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [64]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [65]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [66]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [67]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [68]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [69]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [70]:
result['entities']=result['text'].apply(entity_wordcloud)

In [71]:
result['count_profane']=result['text'].apply(count_profane_words)

In [72]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [73]:
result['contradiction']=result['text'].apply(get_Antonymity_negativity)

Streaming output truncated to the last 5000 lines.
2 . 
False
gave
Antonym: {'take', 'starve'}
WordNum is:14
0 the 
False
1 universe 
False
2 gave 
False
3 me 
False
4 the 
False
5 task 
False
6 of 
False
7 a 
False
8 potty 
False
9 training 
False
10 a 
False
11 whole 
False
12 human 
False
13 ? 
False
WordNum is:3
0 the 
False
1 audacity 
False
2 . 
False
feel
Antonym: set()
WordNum is:19
0 Sometimes 
False
1 I 
False
2 feel 
False
3 like 
False
4 all 
False
5 this 
False
6 effort 
False
7 is 
False
8 just 
False
9 like 
False
10 sticking 
False
11 diamond 
False
12 encrusted 
False
13 alloys 
False
14 on 
False
15 a 
False
16 knackered 
False
17 Corsa ORG
False
18 . 
False
WordNum is:15
0 I 
False
1 mean 
False
2 , 
False
3 shit 
False
4 is 
False
5 what 
False
6 it 
False
7 is 
False
8 at 
False
9 the DATE
False
10 end DATE
False
11 of DATE
False
12 the DATE
False
13 day DATE
False
14 . 
False
WordNum is:5
0 Day 
False
1 55 CARDINAL
True
2 of 
False
3 Lockdown GPE
False
4 . 
False


In [74]:
result['mixed_cases']=result['text'].apply(extract_mixed_cpatilization)

In [75]:
result['quotes']=result['text'].apply(quotes)

In [76]:
result['count_interjections']=result['text'].apply(get_count_interjections)

In [ ]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane,contradiction,mixed_cases,quotes,count_interjections
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0.200000,0.540000,0.204833,0.196862,0.007971,0.000000,0.00000,0.281642,0.169893,0.111748,0.200000,0.540000,9,0.956739,True,1,3,0.680000,2,4,4,2,Ben Platt @DearEvanHansen Grease 1,0,False,7,0,0
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0.020833,0.156250,0.343094,0.245032,0.098062,0.020833,0.15625,0.274382,0.246909,0.027473,0.000000,0.000000,1,0.947632,True,0,0,0.451389,4,0,3,5,three,1,False,0,0,1
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00000,0.229167,0.000000,0.229167,0.250000,0.333333,2,0.875625,False,0,0,0.616667,0,2,0,2,Billy Gunn first years,0,False,2,0,0
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,-0.033333,0.000000,0.046875,0.036458,0.010417,-0.033333,0.00000,0.064560,0.072802,-0.008242,0.000000,0.000000,10,0.917083,True,0,0,0.000000,3,5,1,0,Dettol Toilet Duck Zoflora,0,False,7,0,0
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,-0.483333,0.483333,0.000000,0.000000,0.000000,0.000000,0.00000,0.041667,0.316667,-0.275000,-0.483333,0.483333,2,0.889444,True,0,0,0.533333,2,0,3,2,,0,False,1,1,0


In [ ]:
len(test)

174

In [77]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["text"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), 0,test['text'][i]))

In [78]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [79]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,-0.775758,0.00,0.000000,0.000000,0.791667,-0.791667,0.00,0.000000,0.095455,0.079545,0.015909,0.0,0.0,0,0.119282
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,-0.104167,0.00,0.000000,0.000000,0.010417,-0.010417,0.00,0.000000,0.000000,0.104167,-0.104167,0.0,0.0,1,0.502500
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.791667,0.25,0.347917,0.979167,0.250000,0.729167,0.25,0.347917,0.187500,0.125000,0.062500,0.0,0.0,2,0.067000
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,-0.917411,-0.10,0.400000,0.229167,0.645833,-0.416667,-0.10,0.350000,0.262649,0.763393,-0.500744,-0.1,0.5,6,0.167500
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.125000,0.20,0.200000,0.000000,0.000000,0.000000,0.40,0.400000,0.208333,0.083333,0.125000,0.0,0.0,2,0.219731


In [80]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [81]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [82]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [83]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [84]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [85]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [86]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [87]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [88]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [89]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [90]:
result_test['contradiction']=result_test['text'].apply(get_Antonymity_negativity)

Streaming output truncated to the last 5000 lines.
3 ! 
False
WordNum is:1
0 . 
False
WordNum is:1
0 . 
False
WordNum is:1
0 . 
False
WordNum is:4
0 Dreams 
False
1 are 
False
2 amazing 
False
3 ! 
False
WordNum is:1
0 . 
False
WordNum is:4
0 Dreams 
False
1 are 
False
2 amazing 
False
3 ! 
False
WordNum is:18
0 Love 
False
1 how 
False
2 modern 
False
3 society 
False
4 has 
False
5 the 
False
6 same 
False
7 rules 
False
8 for 
False
9 everyone 
False
10 , 
False
11 regardless 
False
12 of 
False
13 your 
False
14 job 
False
15 or 
False
16 background 
False
17 . 
False
WordNum is:6
0 Really 
False
1 brings 
False
2 the 
False
3 country 
False
4 together 
False
5 ! 
False
given
Antonym: {'take', 'starve'}
stay
Antonym: {'take', 'depart', 'move', 'change', 'starve'}
WordNum is:16
0 So 
False
1 the 
False
2 PM 
False
3 has 
False
4 finally 
False
5 given 
False
6 his 
False
7 verdict 
False
8 on 
False
9 his 
False
10 COVID ORG
False
11 NHS 
False
12 stay 
False
13 in 
False
14 hospita

In [91]:
result_test['mixed_cases']=result_test['text'].apply(extract_mixed_cpatilization)

In [92]:
result_test['quotes']=result_test['text'].apply(quotes)

In [93]:
result_test['count_interjections']=result_test['text'].apply(get_count_interjections)

In [ ]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane,contradiction,mixed_cases,quotes,count_interjections
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,-0.775758,0.00,0.000000,0.000000,0.791667,-0.791667,0.00,0.000000,0.095455,0.079545,0.015909,0.0,0.0,0,0.119282,True,0,0,0.000000,3,0,0,1,the day,0,False,0,1,0
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,-0.104167,0.00,0.000000,0.000000,0.010417,-0.010417,0.00,0.000000,0.000000,0.104167,-0.104167,0.0,0.0,1,0.502500,True,0,0,0.000000,1,0,0,2,,0,False,1,0,0
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.791667,0.25,0.347917,0.979167,0.250000,0.729167,0.25,0.347917,0.187500,0.125000,0.062500,0.0,0.0,2,0.067000,True,0,0,0.502778,4,1,2,4,first first my entire year 1995,0,False,1,0,0
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,-0.917411,-0.10,0.400000,0.229167,0.645833,-0.416667,-0.10,0.350000,0.262649,0.763393,-0.500744,-0.1,0.5,6,0.167500,True,0,1,0.350000,3,0,1,0,,0,False,3,0,1
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.125000,0.20,0.200000,0.000000,0.000000,0.000000,0.40,0.400000,0.208333,0.083333,0.125000,0.0,0.0,2,0.219731,False,0,0,0.000000,1,2,0,1,american,0,False,1,0,0


In [ ]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'entities', 'count_profane', 'contradiction',
       'mixed_cases', 'quotes', 'count_interjections'],
      dtype='object')

In [94]:
result_test.fillna(0,inplace=True)


In [ ]:
result_test.dropna(inplace=True)

In [95]:
result.dropna(inplace=True)

In [96]:
x_train,y_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane', 'mixed_cases','quotes','contradiction','count_interjections']],result[['label']]

In [97]:
x_test,y_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane',  'mixed_cases', 'quotes','contradiction','count_interjections']],result_test[['label']]

In [98]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler()
train_transformed=scaled_features.fit_transform(x_train.values)
test_transformed=scaled_features.transform(x_test.values)

In [99]:
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
train_transformed_PCA=scaled_features.fit_transform(train_transformed)
test_transformed_PCA=scaled_features.transform(test_transformed)

In [ ]:
y_test

,label
0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...
169,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
170,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
171,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
172,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [100]:
from sklearn.multioutput import MultiOutputClassifier

In [101]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [102]:
def print_statistics(y, y_pred):
    accuracy = hamming_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'estimator__C': C_range}
    linear_svm = MultiOutputClassifier(LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2'))
    # print(linear_svm.get_params().keys())
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'estimator__kernel': ['rbf'], 'estimator__C': C_range, 'estimator__gamma': gamma_range}
    nonlinear_svm = MultiOutputClassifier(SVC(class_weight=class_ratio))
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'estimator__C': C_range}
    log_regr = MultiOutputClassifier(LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2'))
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = MultiOutputClassifier(LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2'))
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    df = pd.DataFrame()
    df['pred_0']=y_hat[:,0]
    df['pred_1']=y_hat[:,1]
    df['pred_2']=y_hat[:,2]
    df['pred_3']=y_hat[:,3]
    df['pred_4']=y_hat[:,4]
    df['pred_5']=y_hat[:,5]
    # df_pred = pd.DataFrame(y_hat,columns =['Predictiion'])
    df.to_csv('/content/drive/MyDrive/ISarcasm/ML_task_B_models/svm_model_prediction_ML_FE_4.csv',index=False)
    # print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = MultiOutputClassifier(LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2'))
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = MultiOutputClassifier(RandomForestClassifier(n_estimators=400, random_state=11))
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =MultiOutputClassifier(XGBClassifier(max_depth=6, n_estimators=1000))
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = MultiOutputClassifier(LinearSVC(C=0.1, penalty='l2'))
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42))
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [103]:
y_train_l=[]
for i in range(0,len(y_train.to_numpy())):
  y_train_l.append(y_train.to_numpy()[i][0].astype(int))
y_train_l=np.array(y_train_l)


In [ ]:
y_test_l=[]
for i in range(0,len(y_test.to_numpy())):
  y_test_l.append(y_test.to_numpy()[i][0].astype(int))
y_test_l=np.array(y_test_l)

In [104]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
linear_svm_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.058
Precision: 0.508
Recall: 0.085
F_score: 0.115

                     precision    recall  f1-score   support

            sarcasm       0.60      0.02      0.04       138
              irony       0.20      0.11      0.14        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.69      0.44      0.54        25

          micro avg       0.32      0.09      0.13       211
          macro avg       0.25      0.10      0.12       211
       weighted avg       0.51      0.09      0.12       211
        samples avg       0.09      0.06      0.07       211



In [ ]:
nonlinear_svm_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.697
Precision: 0.577
Recall: 0.649
F_score: 0.584

                     precision    recall  f1-score   support

            sarcasm       0.80      0.99      0.88       138
              irony       0.33      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.78      0.65      0.71       211
          macro avg       0.19      0.17      0.16       211
       weighted avg       0.58      0.65      0.58       211
        samples avg       0.78      0.70      0.73       211



In [ ]:
logistic_regression_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.315
Precision: 0.671
Recall: 0.498
F_score: 0.499

                     precision    recall  f1-score   support

            sarcasm       0.84      0.39      0.53       138
              irony       0.24      0.67      0.35        36
             satire       0.11      0.80      0.20         5
     understatement       0.02      1.00      0.03         1
      overstatement       0.06      0.50      0.10         6
rhetorical_question       0.63      0.76      0.69        25

          micro avg       0.30      0.50      0.38       211
          macro avg       0.32      0.69      0.32       211
       weighted avg       0.67      0.50      0.50       211
        samples avg       0.33      0.47      0.38       211



In [ ]:
linear_svm(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.345
Precision: 0.662
Recall: 0.588
F_score: 0.582

                     precision    recall  f1-score   support

            sarcasm       0.83      0.56      0.67       138
              irony       0.24      0.53      0.33        36
             satire       0.08      0.80      0.14         5
     understatement       0.02      1.00      0.03         1
      overstatement       0.07      0.67      0.13         6
rhetorical_question       0.63      0.76      0.69        25

          micro avg       0.34      0.59      0.43       211
          macro avg       0.31      0.72      0.33       211
       weighted avg       0.66      0.59      0.58       211
        samples avg       0.37      0.57      0.43       211



In [ ]:
logistic_regression(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.350
Precision: 0.648
Recall: 0.569
F_score: 0.567

                     precision    recall  f1-score   support

            sarcasm       0.82      0.55      0.66       138
              irony       0.23      0.53      0.32        36
             satire       0.09      0.60      0.16         5
     understatement       0.02      1.00      0.04         1
      overstatement       0.05      0.67      0.10         6
rhetorical_question       0.59      0.68      0.63        25

          micro avg       0.33      0.57      0.42       211
          macro avg       0.30      0.67      0.32       211
       weighted avg       0.65      0.57      0.57       211
        samples avg       0.37      0.57      0.43       211



In [ ]:
random_forest(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


       Random Forest

Accuracy: 0.722
Precision: 0.637
Recall: 0.682
F_score: 0.624

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.24      0.39        25

          micro avg       0.80      0.68      0.74       211
          macro avg       0.30      0.21      0.21       211
       weighted avg       0.64      0.68      0.62       211
        samples avg       0.80      0.72      0.75       211



In [ ]:
xg_boost(x_train,y_train_l,x_test,y_test_l)


             XGBoost

Accuracy: 0.671
Precision: 0.637
Recall: 0.668
F_score: 0.640

                     precision    recall  f1-score   support

            sarcasm       0.79      0.92      0.85       138
              irony       0.23      0.08      0.12        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.69      0.44      0.54        25

          micro avg       0.74      0.67      0.70       211
          macro avg       0.28      0.24      0.25       211
       weighted avg       0.64      0.67      0.64       211
        samples avg       0.73      0.70      0.70       211



In [ ]:
gradient_boosting(x_train,y_train_l,x_test,y_test_l)


   Gradient Boosting

Accuracy: 0.629
Precision: 0.642
Recall: 0.602
F_score: 0.612

                     precision    recall  f1-score   support

            sarcasm       0.80      0.83      0.81       138
              irony       0.27      0.19      0.23        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.60      0.24      0.34        25

          micro avg       0.71      0.60      0.65       211
          macro avg       0.28      0.21      0.23       211
       weighted avg       0.64      0.60      0.61       211
        samples avg       0.70      0.64      0.66       211



With satndard Scalar

In [ ]:
linear_svm_grid(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.486
Precision: 0.660
Recall: 0.621
F_score: 0.625

                     precision    recall  f1-score   support

            sarcasm       0.81      0.67      0.74       138
              irony       0.23      0.53      0.32        36
             satire       0.17      0.40      0.24         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.74      0.68      0.71        25

          micro avg       0.50      0.62      0.56       211
          macro avg       0.32      0.38      0.33       211
       weighted avg       0.66      0.62      0.63       211
        samples avg       0.53      0.63      0.55       211



In [ ]:
nonlinear_svm_grid(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
logistic_regression_grid(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.352
Precision: 0.691
Recall: 0.555
F_score: 0.550

                     precision    recall  f1-score   support

            sarcasm       0.86      0.46      0.60       138
              irony       0.26      0.72      0.38        36
             satire       0.10      0.80      0.18         5
     understatement       0.02      1.00      0.03         1
      overstatement       0.05      0.50      0.10         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.33      0.55      0.41       211
          macro avg       0.32      0.71      0.33       211
       weighted avg       0.69      0.55      0.55       211
        samples avg       0.37      0.54      0.42       211



In [105]:
linear_svm(train_transformed,y_train_l,test_transformed,_,class_ratio='balanced')


          Linear SVM



In [ ]:
linear_svm(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.289
Precision: 0.689
Recall: 0.526
F_score: 0.514

                     precision    recall  f1-score   support

            sarcasm       0.86      0.41      0.55       138
              irony       0.24      0.72      0.36        36
             satire       0.07      0.80      0.13         5
     understatement       0.01      1.00      0.03         1
      overstatement       0.07      0.83      0.13         6
rhetorical_question       0.68      0.76      0.72        25

          micro avg       0.28      0.53      0.37       211
          macro avg       0.32      0.75      0.32       211
       weighted avg       0.69      0.53      0.51       211
        samples avg       0.31      0.50      0.36       211



In [ ]:
logistic_regression(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.300
Precision: 0.679
Recall: 0.517
F_score: 0.511

                     precision    recall  f1-score   support

            sarcasm       0.85      0.41      0.56       138
              irony       0.24      0.72      0.36        36
             satire       0.08      0.80      0.15         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.07      0.83      0.12         6
rhetorical_question       0.65      0.68      0.67        25

          micro avg       0.29      0.52      0.37       211
          macro avg       0.32      0.57      0.31       211
       weighted avg       0.68      0.52      0.51       211
        samples avg       0.32      0.50      0.37       211



In [ ]:
random_forest(train_transformed,y_train_l,test_transformed,y_test_l,class_ratio='balanced')


       Random Forest

Accuracy: 0.722
Precision: 0.637
Recall: 0.682
F_score: 0.624

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.24      0.39        25

          micro avg       0.80      0.68      0.74       211
          macro avg       0.30      0.21      0.21       211
       weighted avg       0.64      0.68      0.62       211
        samples avg       0.80      0.72      0.75       211



In [ ]:
xg_boost(train_transformed,y_train_l,test_transformed,y_test_l)


             XGBoost

Accuracy: 0.692
Precision: 0.645
Recall: 0.692
F_score: 0.656

                     precision    recall  f1-score   support

            sarcasm       0.80      0.95      0.87       138
              irony       0.23      0.08      0.12        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.71      0.48      0.57        25

          micro avg       0.75      0.69      0.72       211
          macro avg       0.29      0.25      0.26       211
       weighted avg       0.65      0.69      0.66       211
        samples avg       0.76      0.72      0.72       211



In [ ]:
gradient_boosting(train_transformed,y_train_l,test_transformed,y_test_l)


   Gradient Boosting

Accuracy: 0.629
Precision: 0.644
Recall: 0.602
F_score: 0.613

                     precision    recall  f1-score   support

            sarcasm       0.81      0.83      0.82       138
              irony       0.26      0.19      0.22        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.60      0.24      0.34        25

          micro avg       0.71      0.60      0.65       211
          macro avg       0.28      0.21      0.23       211
       weighted avg       0.64      0.60      0.61       211
        samples avg       0.69      0.64      0.66       211



With PCA

In [ ]:
linear_svm_grid(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.522
Precision: 0.615
Recall: 0.583
F_score: 0.597

                     precision    recall  f1-score   support

            sarcasm       0.80      0.77      0.78       138
              irony       0.19      0.11      0.14        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.52      0.52      0.52        25

          micro avg       0.64      0.58      0.61       211
          macro avg       0.25      0.23      0.24       211
       weighted avg       0.62      0.58      0.60       211
        samples avg       0.58      0.59      0.56       211



In [ ]:
nonlinear_svm_grid(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
logistic_regression_grid(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.352
Precision: 0.691
Recall: 0.555
F_score: 0.550

                     precision    recall  f1-score   support

            sarcasm       0.86      0.46      0.60       138
              irony       0.26      0.72      0.38        36
             satire       0.10      0.80      0.18         5
     understatement       0.02      1.00      0.03         1
      overstatement       0.05      0.50      0.10         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.33      0.55      0.41       211
          macro avg       0.32      0.71      0.33       211
       weighted avg       0.69      0.55      0.55       211
        samples avg       0.37      0.54      0.42       211



In [ ]:
linear_svm(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.289
Precision: 0.689
Recall: 0.526
F_score: 0.514

                     precision    recall  f1-score   support

            sarcasm       0.86      0.41      0.55       138
              irony       0.24      0.72      0.36        36
             satire       0.07      0.80      0.13         5
     understatement       0.01      1.00      0.03         1
      overstatement       0.07      0.83      0.13         6
rhetorical_question       0.68      0.76      0.72        25

          micro avg       0.28      0.53      0.37       211
          macro avg       0.32      0.75      0.32       211
       weighted avg       0.69      0.53      0.51       211
        samples avg       0.31      0.50      0.36       211



In [ ]:
logistic_regression(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.300
Precision: 0.679
Recall: 0.517
F_score: 0.511

                     precision    recall  f1-score   support

            sarcasm       0.85      0.41      0.56       138
              irony       0.24      0.72      0.36        36
             satire       0.08      0.80      0.15         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.07      0.83      0.12         6
rhetorical_question       0.65      0.68      0.67        25

          micro avg       0.29      0.52      0.37       211
          macro avg       0.32      0.57      0.31       211
       weighted avg       0.68      0.52      0.51       211
        samples avg       0.32      0.50      0.37       211



In [ ]:
random_forest(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l,class_ratio='balanced')


       Random Forest

Accuracy: 0.722
Precision: 0.637
Recall: 0.682
F_score: 0.624

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       1.00      0.24      0.39        25

          micro avg       0.80      0.68      0.74       211
          macro avg       0.30      0.21      0.21       211
       weighted avg       0.64      0.68      0.62       211
        samples avg       0.80      0.72      0.75       211



In [ ]:
xg_boost(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l)


             XGBoost

Accuracy: 0.692
Precision: 0.645
Recall: 0.692
F_score: 0.656

                     precision    recall  f1-score   support

            sarcasm       0.80      0.95      0.87       138
              irony       0.23      0.08      0.12        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.71      0.48      0.57        25

          micro avg       0.75      0.69      0.72       211
          macro avg       0.29      0.25      0.26       211
       weighted avg       0.65      0.69      0.66       211
        samples avg       0.76      0.72      0.72       211



In [ ]:
gradient_boosting(train_transformed_PCA,y_train_l,test_transformed_PCA,y_test_l)


   Gradient Boosting

Accuracy: 0.629
Precision: 0.644
Recall: 0.602
F_score: 0.613

                     precision    recall  f1-score   support

            sarcasm       0.81      0.83      0.82       138
              irony       0.26      0.19      0.22        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.60      0.24      0.34        25

          micro avg       0.71      0.60      0.65       211
          macro avg       0.28      0.21      0.23       211
       weighted avg       0.64      0.60      0.61       211
        samples avg       0.69      0.64      0.66       211



In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(result['text'])
char_vectorizer.fit(result['text'])


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [ ]:
train_word_features = word_vectorizer.transform(result['text'])
train_char_features = char_vectorizer.transform(result['text'])

In [ ]:
test_word_features = word_vectorizer.transform(result_test['text'])
test_char_features = char_vectorizer.transform(result_test['text'])

In [ ]:
from scipy.sparse import hstack, csr_matrix


In [ ]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [ ]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [ ]:
train_features_transfomred = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(train_transformed, dtype=float))], 'csr'
)

In [ ]:
test_features_transfomred = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(test_transformed, dtype=float))], 'csr'
)

In [ ]:
train_features_transfomred_PCA = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(train_transformed_PCA, dtype=float))], 'csr'
)

In [ ]:
test_features_transfomred_PCA = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(test_transformed_PCA, dtype=float))], 'csr'
)

In [ ]:
linear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.705
Precision: 0.692
Recall: 0.678
F_score: 0.672

                     precision    recall  f1-score   support

            sarcasm       0.82      0.91      0.86       138
              irony       0.35      0.22      0.27        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.83      0.40      0.54        25

          micro avg       0.76      0.68      0.72       211
          macro avg       0.33      0.25      0.28       211
       weighted avg       0.69      0.68      0.67       211
        samples avg       0.76      0.72      0.73       211



In [ ]:
nonlinear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
logistic_regression_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.657
Precision: 0.672
Recall: 0.645
F_score: 0.649

                     precision    recall  f1-score   support

            sarcasm       0.81      0.86      0.83       138
              irony       0.25      0.19      0.22        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.85      0.44      0.58        25

          micro avg       0.71      0.64      0.67       211
          macro avg       0.32      0.25      0.27       211
       weighted avg       0.67      0.64      0.65       211
        samples avg       0.71      0.67      0.68       211



In [ ]:
linear_svm(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.421
Precision: 0.650
Recall: 0.592
F_score: 0.588

                     precision    recall  f1-score   support

            sarcasm       0.81      0.58      0.68       138
              irony       0.23      0.47      0.31        36
             satire       0.16      0.80      0.27         5
     understatement       0.03      1.00      0.06         1
      overstatement       0.11      0.67      0.19         6
rhetorical_question       0.63      0.76      0.69        25

          micro avg       0.42      0.59      0.49       211
          macro avg       0.33      0.71      0.37       211
       weighted avg       0.65      0.59      0.59       211
        samples avg       0.45      0.57      0.48       211



In [ ]:
logistic_regression(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.362
Precision: 0.650
Recall: 0.573
F_score: 0.574

                     precision    recall  f1-score   support

            sarcasm       0.82      0.57      0.67       138
              irony       0.23      0.50      0.32        36
             satire       0.11      0.60      0.18         5
     understatement       0.02      1.00      0.05         1
      overstatement       0.06      0.67      0.11         6
rhetorical_question       0.59      0.68      0.63        25

          micro avg       0.36      0.57      0.44       211
          macro avg       0.30      0.67      0.33       211
       weighted avg       0.65      0.57      0.57       211
        samples avg       0.38      0.57      0.44       211



In [ ]:
random_forest(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
gradient_boosting(train_features,y_train_l,test_features,y_test_l)


   Gradient Boosting

Accuracy: 0.679
Precision: 0.593
Recall: 0.664
F_score: 0.615

                     precision    recall  f1-score   support

            sarcasm       0.79      0.96      0.87       138
              irony       0.14      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.44      0.28      0.34        25

          micro avg       0.73      0.66      0.70       211
          macro avg       0.23      0.21      0.21       211
       weighted avg       0.59      0.66      0.61       211
        samples avg       0.75      0.70      0.71       211



with standard scalar

In [ ]:
linear_svm_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.697
Precision: 0.692
Recall: 0.668
F_score: 0.669

                     precision    recall  f1-score   support

            sarcasm       0.81      0.88      0.85       138
              irony       0.35      0.22      0.27        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.85      0.44      0.58        25

          micro avg       0.75      0.67      0.71       211
          macro avg       0.33      0.26      0.28       211
       weighted avg       0.69      0.67      0.67       211
        samples avg       0.75      0.70      0.71       211



In [ ]:
nonlinear_svm_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
logistic_regression_grid(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.651
Precision: 0.680
Recall: 0.635
F_score: 0.649

                     precision    recall  f1-score   support

            sarcasm       0.81      0.83      0.82       138
              irony       0.30      0.22      0.25        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.85      0.44      0.58        25

          micro avg       0.72      0.64      0.67       211
          macro avg       0.33      0.25      0.28       211
       weighted avg       0.68      0.64      0.65       211
        samples avg       0.70      0.66      0.67       211



In [ ]:
linear_svm(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.379
Precision: 0.694
Recall: 0.564
F_score: 0.572

                     precision    recall  f1-score   support

            sarcasm       0.86      0.50      0.63       138
              irony       0.26      0.69      0.38        36
             satire       0.12      0.60      0.19         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.06      0.50      0.10         6
rhetorical_question       0.68      0.76      0.72        25

          micro avg       0.37      0.56      0.45       211
          macro avg       0.33      0.51      0.34       211
       weighted avg       0.69      0.56      0.57       211
        samples avg       0.40      0.56      0.45       211



In [ ]:
logistic_regression(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.317
Precision: 0.679
Recall: 0.521
F_score: 0.521

                     precision    recall  f1-score   support

            sarcasm       0.84      0.42      0.56       138
              irony       0.25      0.72      0.37        36
             satire       0.09      0.80      0.16         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.05      0.67      0.10         6
rhetorical_question       0.69      0.72      0.71        25

          micro avg       0.30      0.52      0.38       211
          macro avg       0.32      0.55      0.32       211
       weighted avg       0.68      0.52      0.52       211
        samples avg       0.34      0.51      0.39       211



In [ ]:
random_forest(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
gradient_boosting(train_features_transfomred,y_train_l,test_features_transfomred,y_test_l)


   Gradient Boosting

Accuracy: 0.675
Precision: 0.600
Recall: 0.664
F_score: 0.617

                     precision    recall  f1-score   support

            sarcasm       0.80      0.96      0.87       138
              irony       0.14      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.47      0.28      0.35        25

          micro avg       0.74      0.66      0.70       211
          macro avg       0.23      0.21      0.21       211
       weighted avg       0.60      0.66      0.62       211
        samples avg       0.75      0.70      0.71       211



With PCA

In [ ]:
linear_svm_grid(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.697
Precision: 0.694
Recall: 0.668
F_score: 0.668

                     precision    recall  f1-score   support

            sarcasm       0.81      0.89      0.85       138
              irony       0.36      0.22      0.28        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.83      0.40      0.54        25

          micro avg       0.76      0.67      0.71       211
          macro avg       0.34      0.25      0.28       211
       weighted avg       0.69      0.67      0.67       211
        samples avg       0.75      0.70      0.72       211



In [ ]:
nonlinear_svm_grid(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
logistic_regression_grid(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.651
Precision: 0.680
Recall: 0.635
F_score: 0.649

                     precision    recall  f1-score   support

            sarcasm       0.81      0.83      0.82       138
              irony       0.30      0.22      0.25        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.85      0.44      0.58        25

          micro avg       0.72      0.64      0.67       211
          macro avg       0.33      0.25      0.28       211
       weighted avg       0.68      0.64      0.65       211
        samples avg       0.70      0.66      0.67       211



In [ ]:
linear_svm(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.379
Precision: 0.694
Recall: 0.564
F_score: 0.572

                     precision    recall  f1-score   support

            sarcasm       0.86      0.50      0.63       138
              irony       0.26      0.69      0.38        36
             satire       0.12      0.60      0.19         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.06      0.50      0.10         6
rhetorical_question       0.68      0.76      0.72        25

          micro avg       0.37      0.56      0.45       211
          macro avg       0.33      0.51      0.34       211
       weighted avg       0.69      0.56      0.57       211
        samples avg       0.40      0.56      0.45       211



In [ ]:
logistic_regression(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.317
Precision: 0.679
Recall: 0.521
F_score: 0.521

                     precision    recall  f1-score   support

            sarcasm       0.84      0.42      0.56       138
              irony       0.25      0.72      0.37        36
             satire       0.09      0.80      0.16         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.05      0.67      0.10         6
rhetorical_question       0.69      0.72      0.71        25

          micro avg       0.30      0.52      0.38       211
          macro avg       0.32      0.55      0.32       211
       weighted avg       0.68      0.52      0.52       211
        samples avg       0.34      0.51      0.39       211



In [ ]:
random_forest(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [ ]:
gradient_boosting(train_features_transfomred_PCA,y_train_l,test_features_transfomred_PCA,y_test_l)


   Gradient Boosting

Accuracy: 0.675
Precision: 0.600
Recall: 0.664
F_score: 0.617

                     precision    recall  f1-score   support

            sarcasm       0.80      0.96      0.87       138
              irony       0.14      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.47      0.28      0.35        25

          micro avg       0.74      0.66      0.70       211
          macro avg       0.23      0.21      0.21       211
       weighted avg       0.60      0.66      0.62       211
        samples avg       0.75      0.70      0.71       211

